### **1\. ESM 기본 개념과 import/export 문법**  

**🔷 기본 문법: export / import**  
export와 import는 모듈 간의 공용 인터페이스를 정의하는 핵심 문법입니다.  
export는 모듈의 내부 자원(변수, 함수, 클래스 등)을 외부로 공개하고, import는 공개된 자원을 가져와 현재 모듈에서 사용할 수 있게 합니다.

**✔️ Named Export / Import 사용하기**  
Named export는 모듈에서 여러 값을 내보낼 때 사용합니다. 
각 값은 고유한 이름을 가지며, import할 때 정확한 이름을 지정해야 합니다.  

```
// math.js - Named exports
export const PI = 3.14159;

export function add(a, b) {
  return a + b;
}

export function multiply(a, b) {
  return a * b;
}

// 또는 한 번에 export
const E = 2.71828;
function subtract(a, b) {
  return a - b;
}

export { E, subtract };
```


In [1]:
import { PI, add, multiply } from './6.math.js';

console.log(PI); // 3.14159
console.log(add(5, 3)); // 8
console.log(multiply(4, 2)); // 8

// 별칭(alias) 사용
// 모듈은 한 번만 실행되고, 여러 곳에서 import해도 같은 인스턴스를 공유합니다
import { subtract as minus } from './6.math.js';
console.log(minus(10, 3)); // 7

// 모든 export를 객체로 가져오기
// 모듈은 한 번만 실행되고, 여러 곳에서 import해도 같은 인스턴스를 공유합니다
import * as MathUtils from './6.math.js';
console.log(MathUtils.PI); // 3.14159
console.log(MathUtils.add(1, 2)); // 3

3.14159
8
8
7
3.14159
3


**✔️ Default Export / Import 사용하기**  
모듈당 단 1개만 지정할 수 있으며, 이 값은 그 모듈의 대표값으로 간주됩니다. 가져올 때 이름을 자유롭게 지정할 수 있습니다  
```
// calculator.js - Default export
export default class Calculator {
  constructor() {
    this.result = 0;
  }

  add(num) {
    this.result += num;
    return this;
  }

  multiply(num) {
    this.result *= num;
    return this;
  }

  getResult() {
    return this.result;
  }
}

// 함수를 default export
// export default function calculate(x, y) {
//   return x + y;
// }
```


In [3]:
// app.js - Default import
import Calculator from './6.calculator.js';

const calc = new Calculator();
const result = calc.add(10).multiply(2).getResult();
console.log(result); // 20

// 원하는 이름으로 import 가능
import MyCalc from './6.calculator.js';
const calc2 = new MyCalc();

20


**✔️ Named와 Default Export 혼합 사용**  
하나의 모듈에서 대표값은 Default Export로, 부가적인 값들은 Named Export로 제공할 수 있습니다.  
```
// user.js - 혼합 사용
export default class User {
  constructor(name, email) {
    this.name = name;
    this.email = email;
  }
}

export const USER_ROLES = {
  ADMIN: 'admin',
  USER: 'user',
  GUEST: 'guest'
};

export function validateEmail(email) {
  return email.includes('@');
}
```

In [5]:
// main.js - 혼합 import
import User, { USER_ROLES, validateEmail } from './6.user.js';

const admin = new User('Alice', 'alice@example.com');
console.log(admin.name); // Alice
console.log(USER_ROLES.ADMIN); // 'admin'
console.log(validateEmail('test@email.com')); // true

Alice
admin
true


**✔️ Re-export**

다른 모듈에서 가져온 값을 현재 모듈의 인터페이스를 통해 다시 외부에 공개하는 기능입니다. 모듈을 구조화하거나, 여러 모듈을 하나의 패키지로 묶어 제공할 때 유용합니다.  


```
// 6.shapes.circle.js
export class Circle {
  constructor(radius) {
    this.radius = radius;
  }
  
  area() {
    return Math.PI * this.radius ** 2;
  }
}

// 6.shapes.rectangle.js
export class Rectangle {
  constructor(width, height) {
    this.width = width;
    this.height = height;
  }
  
  area() {
    return this.width * this.height;
  }
}

// 6.shapes.index.js - 메인 진입점
export { Circle } from './6.shapes.circle.js';
export { Rectangle } from './6.shapes.rectangle.js';

// 또는 모든 것을 re-export
// export * from './circle.js';
// export * from './rectangle.js';
```

In [6]:
// app.js - 단일 진입점에서 import
import { Circle, Rectangle } from './6.shapes.index.js';

const circle = new Circle(5);
const rect = new Rectangle(4, 6);

console.log(circle.area()); // 78.53981633974483
console.log(rect.area()); // 24

78.53981633974483
24


### **2\. 모듈 로딩과 해석(Resolution), dynamic import**  
모듈을 가져오는 과정은 단순히 파일을 읽는 것이 아니라, 경로 해석(Resolution) → 모듈 로딩(Loading) → 평가(Evaluation)의 단계를 거칩니다.

**✔️ dynamic import() 예시**  
import()는 함수처럼 호출되며 Promise를 반환합니다. 이를 통해 런타임 조건에 따라 필요한 모듈만 비동기적으로 로드할 수 있습니다.


In [ ]:
// 조건부 로딩 예제
let parser;
if (mode === 'json') {
  // Promise를 await으로 기다려 모듈 객체를 얻음
  parser = await import('./jsonParser.js');
} else {
  parser = await import('./xmlParser.js');
}

// 모듈 객체를 통해 export된 함수 사용
parser.parse(data);

// 모듈 경로를 동적으로 구성하여 로드
const locale = navigator.language;
const messages = await import(`./i18n/${locale}.js`);

**📌 모듈 캐싱 (Memoization)**  
모듈은 최초 로딩 후 모듈 인스턴스가 캐싱됩니다. 두 번째 import는 파일을 재평가하지 않고 캐시된 동일한 인스턴스를 반환합니다.


In [ ]:
// math.js 파일을 첫 번째로 로딩 및 평가
const a1 = await import('./6.math.js');
// math.js 파일을 두 번째로 로딩 (캐시된 인스턴스를 반환)
const a2 = await import('./6.math.js');

console.log(a1 === a2); // 출력: true (동일 객체)

true


### **3\. Top-Level await (ES2022)**  

**✔️ Top-Level await란?**  
ES2022에서 도입된 Top-Level await는 모듈의 최상위 레벨에서 async 함수 없이 await를 직접 사용할 수 있게 해줍니다.  
이는 모듈 초기화 과정을 크게 단순화합니다.

**✔️ 동적 설정 및 환경 로딩**

애플리케이션 시작 시점에 환경(운영, 개발 등)을 감지하고, 해당 환경에 맞는 설정 모듈을 동적으로 로드하여 내보내는 패턴입니다.


In [ ]:
// config-loader.js
// 1. 환경 변수를 비동기로 로드
const envResponse = await fetch('/api/environment');
const environment = await envResponse.json();

// 2. 환경에 따라 다른 설정 파일을 동적 import 및 await
let config;
if (environment.mode === 'production') {
  config = await import('./config.prod.js');
} else {
  config = await import('./config.dev.js');
}

// 3. 데이터베이스 연결 등 후속 초기화 작업 수행
const db = await initializeDatabase(config.dbUrl);

// 모든 것이 준비된 후 export
export const appConfig = config.default;
export const database = db;

// main.js: config-loader.js의 모든 비동기 작업이 끝난 후 실행 시작
import { appConfig, database } from './config-loader.js';
console.log('App started with config:', appConfig);

**✔️ 의존성이 있는 모듈의 순차적 초기화**  
모듈 A가 모듈 B에 의존하고, 모듈 B가 초기화에 비동기 작업을 필요로 할 때, TLA는 실행 순서를 자동으로 보장합니다.  
실행 순서: database.js 완료 → cache.js 완료 → api.js 실행.

In [ ]:
// database.js
console.log('1. Connecting to database...');
const connection = await connectToDatabase();
export const db = connection;

// cache.js (database.js에 의존)
import { db } from './database.js'; // 📌 database.js의 await이 끝날 때까지 대기
console.log('2. Initializing cache with DB connection...');
const cache = await initializeCache(db);
export const appCache = cache;

// api.js (cache.js에 의존)
import { appCache } from './cache.js'; // 📌 cache.js의 await이 끝날 때까지 대기
console.log('3. Starting API server...');
const server = await startServer(appCache);
export const api = server;

**✔️ 조건부 기능 로딩 및 코드 스플리팅**  
기능 플래그나 사용자 권한에 따라 필요한 모듈만 로드하여 초기 로딩 시간을 단축할 수 있습니다.


In [ ]:
// feature-flags.js
const flagsResponse = await fetch('/api/feature-flags');
const flags = await flagsResponse.json();

// 기능 플래그가 true일 때만 해당 모듈을 로드
export const analytics = flags.analyticsEnabled
  ? await import('./analytics.js') // 로드 후 await
  : null;

export const experimentalFeatures = flags.experimental
  ? await import('./experimental.js')
  : null;

// app.js
import { analytics, experimentalFeatures } from './feature-flags.js';

if (analytics) {
  analytics.trackPageView(); // 로드되었을 때만 실행
}

**✔️ 병렬 로딩을 통한 성능 최적화** 
TLA를 사용할 때, 여러 개의 비동기 작업을 순차적으로 await하면 총 초기화 시간이 길어집니다.
Promise.all()을 사용하여 병렬로 실행함으로써 성능을 최적화할 수 있습니다.

In [ ]:
// parallel-loader.js

// ❌ 순차적 로딩 (느림)
// const users = await fetchUsers();
// const products = await fetchProducts();

// ✅ 병렬 로딩 (빠름)
const [users, products, categories] = await Promise.all([
  fetch('/api/users').then(r => r.json()),
  fetch('/api/products').then(r => r.json()),
  fetch('/api/categories').then(r => r.json())
]);

console.log('All data loaded:', {
  userCount: users.length,
  productCount: products.length
});
export { users, products, categories };